<a href="https://colab.research.google.com/github/MadhuryaPasan/Qwen3-ToolCalling-FineTune/blob/main/Notebook1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.3/432.3 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.5/376.5 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 94.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from unsloth import FastVisionModel
import torch

model, tokenizer = FastVisionModel.from_pretrained(
    "unsloth/Qwen3-VL-2B-Instruct-FP8",
    load_in_4bit = False,
    load_in_8bit = True,
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for long context
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.2.1: Fast Qwen3_Vl patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.35. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/269 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/782 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

video_preprocessor_config.json:   0%|          | 0.00/817 [00:00<?, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

In [3]:
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # False if not finetuning vision layers
    finetune_language_layers   = True, # False if not finetuning language layers
    finetune_attention_modules = True, # False if not finetuning attention layers
    finetune_mlp_modules       = True, # False if not finetuning MLP layers

    r = 8,
    lora_alpha = 8,
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None
)

In [4]:
from datasets import load_dataset
dataset = load_dataset("Salesforce/xlam-function-calling-60k", split = "train")

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

xlam_function_calling_60k.json:   0%|          | 0.00/96.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/60000 [00:00<?, ? examples/s]

In [5]:
dataset

Dataset({
    features: ['id', 'query', 'answers', 'tools'],
    num_rows: 60000
})

In [6]:
dataset[1]

{'id': 1,
 'query': "I need to understand the details of the Ethereum blockchain for my cryptocurrency project. Can you fetch the details for 'ethereum'?",
 'answers': '[{"name": "web_chain_details", "arguments": {"chain_slug": "ethereum"}}]',
 'tools': '[{"name": "peers", "description": "Retrieves a list of company peers given a stock symbol.", "parameters": {"symbol": {"description": "The stock symbol for the company.", "type": "str", "default": ""}}}, {"name": "web_chain_details", "description": "python", "parameters": {"chain_slug": {"description": "The slug identifier for the blockchain (e.g., \'ethereum\' for Ethereum mainnet).", "type": "str", "default": "ethereum"}}}]'}

In [7]:
# @title
# def convert_xlam_to_conversation(sample):
#     # 1. Define the Standard xLAM System Instruction
#     system_instruction = (
#         "You are an expert in tool calling. You will be provided with a list of tools and a user query. Your goal is to output the correct tool call in JSON format."
#     )

#     # 2. Format the User prompt with clear delimiters
#     # Standard xLAM uses [BEGIN OF TOOL_DEFINITION] or similar structures
#     user_prompt = (
#         f"[BEGIN OF TOOL_DEFINITION]\n{sample['tools']}\n[END OF TOOL_DEFINITION]\n"
#         f"Query: {sample['query']}"
#     )

#     # 3. Format the Assistant output
#     # The 'answers' column in xLAM is already a JSON string
#     assistant_output = sample['answers']

#     return {
#         "messages": [
#             {"role": "system", "content": system_instruction},
#             {"role": "user", "content": user_prompt},
#             {"role": "assistant", "content": assistant_output}
#         ]
#     }

# converted_dataset = dataset.map(convert_xlam_to_conversation,remove_columns=dataset.column_names)

In [8]:
# @title
# from unsloth.chat_templates import get_chat_template

# tokenizer = get_chat_template(
#     tokenizer,
#     chat_template = "qwen-3", # Qwen3 uses the same underlying ChatML structure
# )

# def formatting_prompts_func(sample):
#     # Standard xLAM setup for an Instruct model
#     messages = [
#         {"role": "system", "content": "You are a helpful assistant with access to tools. Output JSON tool calls."},
#         {"role": "user", "content": f"Tools:\n{sample['tools']}\n\nQuery: {sample['query']}"},
#         {"role": "assistant", "content": sample['answers']}
#     ]
#     return {"text": tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)}

# # Apply to your xLAM dataset
# converted_dataset = dataset.map(formatting_prompts_func)

In [10]:
# converted_dataset[1]

In [11]:
from unsloth.chat_templates import get_chat_template

# 1. Initialize the tokenizer with the correct Qwen3 template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen-3", # Use the built-in unsloth qwen-3 template
)

def formatting_prompts_func(sample):
    import json

    tools = json.loads(sample["tools"])
    ans_list = json.loads(sample["answers"])

    tool_calls = []
    for a in ans_list:
        tool_calls.append({
            "type": "function",
            "function": {
                "name": a["name"],
                "arguments": json.dumps(a["arguments"])  # <-- must be a string, not dict
            }
        })

    messages = [
        {"role": "user", "content": sample["query"]},
        {
            "role": "assistant",
            "content": "",
            "tool_calls": tool_calls
        }
    ]

    text = tokenizer.apply_chat_template(
        messages,
        tools=tools,
        tokenize=False,
        add_generation_prompt=False,
    )

    return {"text": text}

dataset = dataset.map(formatting_prompts_func, batched=False)

Map:   0%|          | 0/60000 [00:00<?, ? examples/s]

In [12]:
dataset["text"][1]

'<|im_start|>system\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>\n{"name": "peers", "description": "Retrieves a list of company peers given a stock symbol.", "parameters": {"symbol": {"description": "The stock symbol for the company.", "type": "str", "default": ""}}}\n{"name": "web_chain_details", "description": "python", "parameters": {"chain_slug": {"description": "The slug identifier for the blockchain (e.g., \'ethereum\' for Ethereum mainnet).", "type": "str", "default": "ethereum"}}}\n</tools>\n\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\n<tool_call>\n{"name": <function-name>, "arguments": <args-json-object>}\n</tool_call><|im_end|>\n<|im_start|>user\nI need to understand the details of the Ethereum blockchain for my cryptocurrency project. Can you fetch the details for \'ethereum\'?<|im

In [13]:
from trl import SFTTrainer, SFTConfig

FastVisionModel.for_training(model) # Enable for training!

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    ataset_text_field = "text",
    max_seq_length = 2048,
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 10,
        # num_train_epochs = 1,
        learning_rate = 5e-5,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",     # For Weights and Biases
        remove_unused_columns = True,
        packing = False
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=5):   0%|          | 0/60000 [00:00<?, ? examples/s]

In [14]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 60,000 | Num Epochs = 1 | Total steps = 10
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 8,716,288 of 2,136,248,320 (0.41% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.299600
2,2.422400
3,2.137900
4,2.365300
5,2.182900
6,2.642300
7,2.305400
8,2.474200
9,1.953300
10,2.181000


In [15]:
# 570

In [18]:
!apt-get install zstd -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  zstd
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 603 kB of archives.
After this operation, 1,695 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 zstd amd64 1.4.8+dfsg-3build1 [603 kB]
Fetched 603 kB in 1s (925 kB/s)
Selecting previously unselected package zstd.
(Reading database ... 121852 files and directories currently installed.)
Preparing to unpack .../zstd_1.4.8+dfsg-3build1_amd64.deb ...
Unpacking zstd (1.4.8+dfsg-3build1) ...
Setting up zstd (1.4.8+dfsg-3build1) ...
Processing triggers for man-db (2.10.2-1) ...


In [19]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading ollama-linux-amd64.tar.zst
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [20]:
!nohup ollama serve > ollama.log 2>&1 &

In [21]:
!apt-get install -y pciutils lshw

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpci3 pci.ids usb.ids
The following NEW packages will be installed:
  libpci3 lshw pci.ids pciutils usb.ids
0 upgraded, 5 newly installed, 0 to remove and 37 not upgraded.
Need to get 883 kB of archives.
After this operation, 3,256 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 pci.ids all 0.0~2022.01.22-1ubuntu0.1 [251 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpci3 amd64 1:3.7.0-6 [28.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 lshw amd64 02.19.git.2021.06.19.996aaad9c7-2build1 [321 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 pciutils amd64 1:3.7.0-6 [63.6 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/main amd64 usb.ids all 2022.04.02-1 [219 kB]
Fetched 883 kB in 1s (1,330 kB/s)
Selecting previously unselected package 

In [22]:
!ollama --version

ollama version is 0.16.3


In [34]:
# print(dataset[0])
# print(type(dataset[0]["text"]))

{'id': 0, 'query': 'Where can I find live giveaways for beta access and games?', 'answers': '[{"name": "live_giveaways_by_type", "arguments": {"type": "beta"}}, {"name": "live_giveaways_by_type", "arguments": {"type": "game"}}]', 'tools': '[{"name": "live_giveaways_by_type", "description": "Retrieve live giveaways from the GamerPower API based on the specified type.", "parameters": {"type": {"description": "The type of giveaways to retrieve (e.g., game, loot, beta).", "type": "str", "default": "game"}}}]', 'text': '<|im_start|>system\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>\n{"name": "live_giveaways_by_type", "description": "Retrieve live giveaways from the GamerPower API based on the specified type.", "parameters": {"type": {"description": "The type of giveaways to retrieve (e.g., game, loot, beta).", "type": "str", "default": "game"}}}\n</tools>\n\nFor each func

In [35]:
from transformers import TrainingArguments
import json

# Test if training args are serializable
args_dict = trainer.args.to_dict()
try:
    json.dumps(args_dict)
    print("TrainingArguments OK")
except TypeError as e:
    print(f"Problem in TrainingArguments: {e}")
    for k, v in args_dict.items():
        try:
            json.dumps(v)
        except TypeError:
            print(f"  ❌ Non-serializable key: {k} = {v}")

TrainingArguments OK


In [36]:
import json

config_dict = model.config.to_dict()

for k, v in config_dict.items():
    try:
        json.dumps(v)
    except TypeError:
        print(f"❌ Non-serializable key: {k} = {v}")

❌ Non-serializable key: quantization_config = {'quant_method': <QuantizationMethod.BITS_AND_BYTES: 'bitsandbytes'>, '_load_in_8bit': True, '_load_in_4bit': False, 'llm_int8_threshold': 6.0, 'llm_int8_skip_modules': ['lm_head', 'multi_modal_projector', 'merger', 'modality_projection', 'router', 'mlp.gate', 'block_sparse_moe.gate', 'mamba', 'audio_tower', 'vision_tower'], 'llm_int8_enable_fp32_cpu_offload': False, 'llm_int8_has_fp16_weight': False, 'bnb_4bit_quant_type': 'fp4', 'bnb_4bit_use_double_quant': False, 'bnb_4bit_compute_dtype': 'float32', 'bnb_4bit_quant_storage': 'uint8', 'get_loading_attributes': <function FastBaseModel.from_pretrained.<locals>.<lambda> at 0x7c0e9a4dd1c0>, 'load_in_4bit': False, 'load_in_8bit': True}


In [33]:
print(type(tokenizer.chat_template))  # if this says <class 'function'>, that's your problem
print(tokenizer.chat_template)

<class 'str'>

{%- if tools %}
    {{- '<|im_start|>system
' }}
    {%- if messages[0].role == 'system' %}
        {{- messages[0].content + '

' }}
    {%- endif %}
    {{- "# Tools

You may call one or more functions to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>" }}
    {%- for tool in tools %}
        {{- "
" }}
        {{- tool | tojson }}
    {%- endfor %}
    {{- "
</tools>

For each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:
<tool_call>
{\"name\": <function-name>, \"arguments\": <args-json-object>}
</tool_call><|im_end|>
" }}
{%- else %}
    {%- if messages[0].role == 'system' %}
        {{- '<|im_start|>system
' + messages[0].content + '<|im_end|>
' }}
    {%- endif %}
{%- endif %}
{%- set ns = namespace(multi_step_tool=true, last_query_index=messages|length - 1) %}
{%- for forward_message in messages %}
    {%- set index = (messages|length - 1

In [27]:
# !pip install unsloth[colab] --no-deps
# !pip install --upgrade --no-cache-dir unsloth


In [31]:
# model.save_pretrained("lora_model") # Saves only adapter
# # Or for merging:
# model.save_pretrained_merged("model_final", tokenizer,)


In [37]:
# model.save_pretrained_merged("output_dir", tokenizer, save_method="merged_16bit")

In [30]:
# tokenizer.chat_template = tokenizer.chat_template
# tokenizer.save_pretrained("output_dir")

[]

In [38]:
# Remove BnB config artifact
del model.config.quantization_config

In [39]:
model.save_pretrained_gguf("qwen_finetune", tokenizer,quantization_method = "q8_0",)

Unsloth: Merging model weights to 16-bit format...
Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...


Unsloth: Copying 1 files from cache to `qwen_finetune`: 100%|██████████| 1/1 [02:05<00:00, 125.32s/it]


Successfully copied all 1 files from cache to `qwen_finetune`
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Merging weights into 16bit: 100%|██████████| 1/1 [02:30<00:00, 150.21s/it]


Unsloth: Merge process complete. Saved to `/content/qwen_finetune`
Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF f16 might take 3 minutes.
\        /    [2] Converting GGUF f16 to ['q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: Updating system package directories
Unsloth: All required system packages already installed!
Unsloth: Install llama.cpp and building - please wait 1 to 3 minutes
Unsloth: Cloning llama.cpp repository
Unsloth: Install GGUF and other packages
Unsloth: Successfully installed llama.cpp!
Unsloth: Preparing converter script...


Unsloth: [1] Converting model into f16 GGUF format.
This might take 3 minutes...


KeyboardInterrupt: 

In [ ]:
model.push_to_hub_gguf("HF_USERNAME/llama_finetune", tokenizer, token = "...")

In [ ]:
import json
import os
from transformers import AutoConfig

# 1. Create a totally clean, static version of the config
# This removes all the "live" Python logic from Qwen3-VL
raw_config_dict = model.config.to_dict()

def make_serializable(d):
    """Deep-cleans a dictionary of anything that isn't simple data."""
    new_dict = {}
    for k, v in d.items():
        if isinstance(v, dict):
            new_dict[k] = make_serializable(v)
        elif isinstance(v, (str, int, float, bool, list)) or v is None:
            new_dict[k] = v
        else:
            # Skip functions, methods, and tensors
            continue
    return new_dict

clean_config_dict = make_serializable(raw_config_dict)

# 2. Inject the clean dict back into the model config
# We manually override the save function to use our clean data
def manual_save_json(path, use_diff=True):
    with open(path, "w") as f:
        json.dump(clean_config_dict, f, indent=2)

model.config.to_json_file = manual_save_json

# 3. Fix the chat template (the most common function culprit)
if hasattr(tokenizer, "chat_template"):
    tokenizer.chat_template = "<|im_start|>{{role}}\n{{content}}<|im_end|>\n"

# 4. Try saving again
print("🚀 Attempting forced local save...")
try:
    model.save_pretrained_merged("qwen3_final_merged", tokenizer, save_method = "merged_16bit")
    print("✅ SUCCESS! Your model is saved in the folder: qwen3_final_merged")
except Exception as e:
    print(f"❌ Failed again. Error: {e}")

🚀 Attempting forced local save...
Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...


Unsloth: Copying 1 files from cache to `qwen3_final_merged`: 100%|██████████| 1/1 [01:43<00:00, 103.32s/it]


Successfully copied all 1 files from cache to `qwen3_final_merged`
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Merging weights into 16bit: 100%|██████████| 1/1 [02:31<00:00, 151.91s/it]


Unsloth: Merge process complete. Saved to `/content/qwen3_final_merged`
✅ SUCCESS! Your model is saved in the folder: qwen3_final_merged


In [ ]:
# from unsloth import FastLanguageModel

# # Load from the physical folder we just created
# model_gguf, tokenizer_gguf = FastLanguageModel.from_pretrained("qwen3_final_merged")

# model_gguf.push_to_hub_gguf(
#     "MadhuryaPasan/qwen3-2b-tools-on-steroid-v0",
#     tokenizer_gguf,
#     quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
#     token = ""
# )

==((====))==  Unsloth 2026.2.1: Fast Qwen3_Vl patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Converting model to GGUF format...
Unsloth: Merging model weights to 16-bit format...


/usr/local/lib/python3.12/dist-packages/unsloth_zoo/saving_utils.py:1678: UserWarning: Model is not a PeftModel (no Lora adapters detected). Skipping Merge. Please use save_pretrained() or push_to_hub() instead!
  warnings.warn("Model is not a PeftModel (no Lora adapters detected). Skipping Merge. Please use save_pretrained() or push_to_hub() instead!")


Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF f16 might take 3 minutes.
\        /    [2] Converting GGUF f16 to ['q4_k_m', 'q8_0', 'q5_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: Updating system package directories
Unsloth: All required system packages already installed!
Unsloth: Install llama.cpp and building - please wait 1 to 3 minutes
Unsloth: Cloning llama.cpp repository
Unsloth: Install GGUF and other packages
Unsloth: Successfully installed llama.cpp!
Unsloth: Preparing converter script...


Unsloth: [1] Converting model into f16 GGUF format.
This might take 3 minutes...


RuntimeError: Failed to convert model to GGUF: Unsloth: GGUF conversion failed: Unsloth: `config.json` does not exist inside `/tmp/unsloth_gguf_p_0yghlu`.

In [ ]:
# Push the cleaned, merged model to HF in 16-bit first
model_gguf.push_to_hub(
    "MadhuryaPasan/qwen3-2b-tools-on-steroid-v0",
    token = "",
    save_method = "merged_16bit", # Important!
)
tokenizer_gguf.push_to_hub("MadhuryaPasan/qwen3-2b-tools-on-steroid-v0", token = "")

README.md:   0%|          | 0.00/537 [00:00<?, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...roid-v0/model.safetensors:   0%|          | 7.38MB / 1.60GB            

Saved model to https://huggingface.co/MadhuryaPasan/qwen3-2b-tools-on-steroid-v0


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...steroid-v0/tokenizer.json: 100%|##########| 11.4MB / 11.4MB            